## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-23-20-53-19 +0000,nypost,Unhinged skateboarder attacks life-sized Marve...,https://nypost.com/2026/01/23/us-news/marvel-c...
1,2026-01-23-20-53-11 +0000,nyt,Trump’s Turnabout on Greenland Shows the Limit...,https://www.nytimes.com/2026/01/23/us/politics...
2,2026-01-23-20-52-00 +0000,wsj,Treasury Yields Fall to End Week Little Changed,https://www.wsj.com/finance/jgb-futures-consol...
3,2026-01-23-20-49-18 +0000,nypost,Nurses union lowers demand for raises at Mount...,https://nypost.com/2026/01/23/us-news/nyc-nurs...
4,2026-01-23-20-46-39 +0000,cbc,Prime Minister Mark Carney to speak after cabi...,https://www.cbc.ca/news/politics/livestory/mar...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2488/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,82
141,new,24
140,ice,19
15,greenland,18
69,up,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
15,2026-01-23-20-23-54 +0000,nypost,Former Eric Adams deputy Kaz Daughtry nabs job...,https://nypost.com/2026/01/23/us-news/eric-ada...,143
273,2026-01-23-02-49-00 +0000,wsj,A resolution to rein in President Trump’s powe...,https://www.wsj.com/politics/policy/gop-lawmak...,139
98,2026-01-23-17-54-06 +0000,nypost,Ryan Wedding’s is 6th top ten most wanted fugi...,https://nypost.com/2026/01/23/us-news/ryan-wed...,138
275,2026-01-23-02-31-45 +0000,cbc,Trump says he's withdrawing invitation for Car...,https://www.cbc.ca/news/politics/trump-board-o...,136
258,2026-01-23-03-44-07 +0000,nypost,Trump withdraws Canadian PM Mark Carney’s Boar...,https://nypost.com/2026/01/22/us-news/trump-wi...,134


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
15,143,2026-01-23-20-23-54 +0000,nypost,Former Eric Adams deputy Kaz Daughtry nabs job...,https://nypost.com/2026/01/23/us-news/eric-ada...
31,60,2026-01-23-19-52-49 +0000,nypost,Alleged psycho surgeon Michael McKee gives sto...,https://nypost.com/2026/01/23/us-news/alleged-...
273,57,2026-01-23-02-49-00 +0000,wsj,A resolution to rein in President Trump’s powe...,https://www.wsj.com/politics/policy/gop-lawmak...
98,56,2026-01-23-17-54-06 +0000,nypost,Ryan Wedding’s is 6th top ten most wanted fugi...,https://nypost.com/2026/01/23/us-news/ryan-wed...
258,52,2026-01-23-03-44-07 +0000,nypost,Trump withdraws Canadian PM Mark Carney’s Boar...,https://nypost.com/2026/01/22/us-news/trump-wi...
280,44,2026-01-23-02-20-00 +0000,wsj,California Gov. Gavin Newsom spent his days at...,https://www.wsj.com/politics/policy/the-newsom...
211,44,2026-01-23-10-46-56 +0000,nyt,"After Trump’s Ultimatum, Greenland Talks Inclu...",https://www.nytimes.com/2026/01/22/world/europ...
178,40,2026-01-23-13-26-57 +0000,nyt,White House Posts Altered Photo Showing Arrest...,https://www.nytimes.com/2026/01/22/us/politics...
252,40,2026-01-23-05-17-48 +0000,bbc,TikTok closes deal to split US app from global...,https://www.bbc.com/news/articles/c3edd1l328lo...
17,36,2026-01-23-20-21-13 +0000,nypost,Here’s when ‘catastrophic’ winter storm arrive...,https://nypost.com/2026/01/23/us-news/heres-wh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
